In [20]:
## written by I/Users/isabelmetzger/Downloads/tzger
# last updated May 21, 2023

# this jupyter notebook calls data from performance.gov and merges them into a table for further downstream analysis + data products (e.g., Tableau Dashboard)

import pandas as pd
import requests
import datetime
import warnings
from IPython.display import display, HTML
import time
# adding simple warning filter
warnings.simplefilter('ignore')



# adding date
today = datetime.datetime.today().strftime('%Y-%m-%d')
print(f"{today}")

2023-05-21


In [21]:
# creating function for flattening json/dictionary

def flatten_json(y):
    """
    this flattens_json obj
    adds begining key to beginning
    """
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out




In [114]:
base_url = "https://www.performance.gov/cx/data/"

data2definition = {
"service_providers" : "A list of High Impact Service Providers (HISPs).",
"services" : "A list of Services. Each HISP has one or more Services.",
"collections" : "HISPs are required to report their customer feedback for each service quarterly; this file includes one record for each quarterly report",
"omb_cx_reporting_collections" : "This file includes one or more Report Detail records per Data Collection",
}

In [115]:
data_dict = {}

for k in data2definition.keys():
    data_url = f"{base_url}{k}.json"
    print("calling", data_url)
    print()
    data_ = requests.get(data_url).json()
    time.sleep(5)
    data_dict[k] = data_


calling https://www.performance.gov/cx/data/service_providers.json

calling https://www.performance.gov/cx/data/services.json

calling https://www.performance.gov/cx/data/collections.json

calling https://www.performance.gov/cx/data/omb_cx_reporting_collections.json



In [122]:
omb_df = pd.DataFrame(flatten_json(a) for a in data_dict['omb_cx_reporting_collections']['data'])
omb_df.head()

,id,type,attributes_organization_id,attributes_organization_name,attributes_organization_abbreviation,attributes_service_provider_id,attributes_service_provider_name,attributes_collection_id,attributes_collection_name,attributes_collection_year,...,attributes_q11_3,attributes_q11_4,attributes_q11_5,attributes_q11_point_scale,attributes_created_at,attributes_updated_at,attributes_operational_metrics,attributes_service_id,attributes_service_name,attributes_service_slug
0,1,omb_cx_reporting_collections,2129,Department of Veterans Affairs,VA,21,Veterans Health Administration,1,CX Quarterly Reporting,2021,...,0,0,0,NaN,2021-05-13T23:05:19.090Z,2021-08-04T19:44:05.084Z,,2,Receiving outpatient services,va-vha
1,2,omb_cx_reporting_collections,2111,Department of Education,ED,15,Federal Student Aid,2,CX Quarterly Reporting,2021,...,0,0,0,NaN,2021-05-14T05:25:07.624Z,2021-08-04T19:21:24.493Z,,22,Applying for Financial Aid,ed-fsa
2,3,omb_cx_reporting_collections,2111,Department of Education,ED,15,Federal Student Aid,2,CX Quarterly Reporting,2021,...,0,0,0,NaN,2021-05-14T05:33:29.668Z,2021-08-04T19:21:18.048Z,,22,Applying for Financial Aid,ed-fsa
3,4,omb_cx_reporting_collections,2106,Office of Personnel Management,OPM,13,Retirement Services,3,CX Quarterly Reporting,2021,...,0,0,0,NaN,2021-05-14T06:09:22.216Z,2021-08-09T05:16:06.129Z,,18,Applying for retirement benefits,opm-rs
4,5,omb_cx_reporting_collections,2106,Office of Personnel Management,OPM,13,Retirement Services,3,CX Quarterly Reporting,2021,...,0,0,0,NaN,2021-05-14T06:17:22.773Z,2021-08-04T19:47:06.723Z,,18,Applying for retirement benefits,opm-rs


In [146]:

collection_cols = [
     'collection_id',
     'start_date',
     'end_date',
     'year',
     'quarter',
     'integrity_hash',
     'aasm_state',
     'reflection',
     'rating',
     'user_id',
     'updated_at_collection',
    'number_relationship_links'
                  ]

collection_df = pd.DataFrame( data_dict['collections']['data'])
collection_df = collection_df.drop(columns=['attributes']).merge(collection_df['attributes'].apply(pd.Series), right_index=True, left_index=True)

collection_df['collection_id'] = collection_df['id'].map(str)

collection_df['number_relationship_links'] = collection_df['relationships'].map(lambda x: x.get('omb_cx_reporting_collections', {}).get('data', [])).map(len)
display(collection_df.head())                                                                     

,id,type,relationships,name,start_date,end_date,organization_id,organization_name,organization_abbreviation,year,...,service_provider_id,service_provider_name,integrity_hash,aasm_state,reflection,rating,user_id,updated_at,collection_id,number_relationship_links
0,1,collections,{'omb_cx_reporting_collections': {'data': [{'i...,CX Quarterly Reporting,2021-01-01,2021-03-31,2129,Department of Veterans Affairs,VA,2021,...,21,Veterans Health Administration,None,published,The insights collected digitally enabled veter...,TRUE,1,2021-08-04T16:46:59.914Z,1,1
1,2,collections,{'omb_cx_reporting_collections': {'data': [{'i...,CX Quarterly Reporting,2021-01-01,2021-03-31,2111,Department of Education,ED,2021,...,15,Federal Student Aid,None,published,For the FAFSA.gov customer satisfaction survey...,TRUE,87,2021-08-04T16:41:37.845Z,2,2
2,3,collections,{'omb_cx_reporting_collections': {'data': [{'i...,CX Quarterly Reporting,2021-01-01,2021-03-31,2106,Office of Personnel Management,OPM,2021,...,13,Retirement Services,None,published,"For Retirement Services:\r\nOverall, Q2 percen...",TRUE,1,2021-08-04T16:59:16.957Z,3,2
3,4,collections,{'omb_cx_reporting_collections': {'data': [{'i...,CX Quarterly Reporting,2021-01-01,2021-03-31,2106,Office of Personnel Management,OPM,2021,...,12,Federal Employment Services,None,published,USAJOBS Desktop (Summary FY 2021 Q2)\r\n\r\nWh...,TRUE,1,2021-08-04T16:59:39.986Z,4,2
4,6,collections,{'omb_cx_reporting_collections': {'data': [{'i...,CX Quarterly Reporting,2021-01-01,2021-03-31,2098,Department of Agriculture,USDA,2021,...,5,Recreation.gov,None,published,In the second quarter of FY2021 (January - Mar...,TRUE,1,2021-08-04T16:40:22.666Z,6,1


In [171]:
services = pd.DataFrame(d for d in data_dict['services']['data'])
services_df = pd.concat([services[['id', 'type']], services['attributes'].apply(pd.Series)], axis=1)
display(services_df.head(2))

service_providers_df = pd.DataFrame(flatten_json(a) for a in data_dict['service_providers']['data'])
serviceproviderid2desc = service_providers_df.set_index('id')['attributes_description'].to_dict()
display(service_providers_df.head(2))

serviceid2desc = services_df.set_index('id')['description'].to_dict()

,id,type,name,description,organization_id,organization_abbreviation,organization_name,service_provider_id,service_provider_name,justification_text,...,service_slug,service_owner_email,service_managers,url,homepage_url,channels,tags,available_in_person,available_digitally,available_via_phone
0,1,services,Touchpoints,Making customer feedback easier for federal ag...,1,GSA,General Services Administration,44.0,Non-HISP Service Provider,,...,,ryan.wold@gsa.gov,"[{'email': 'philip.ashlock@gsa.gov', 'first_na...",https://touchpoints.digital.gov/,https://touchpoints.digital.gov/,"[{'id': 315, 'name': 'email', 'created_at': '2...","[{'id': 218, 'name': 'surveys', 'created_at': ...",False,True,True
1,2,services,Receiving outpatient services,VHA sends email surveys to everyone who receiv...,2129,VA,Department of Veterans Affairs,21.0,Veterans Health Administration,,...,va-vha,aaron.m.stienstra@omb.eop.gov,[],,None,[],[],False,False,False


,id,type,attributes_organization_id,attributes_organization_abbreviation,attributes_organization_name,attributes_name,attributes_slug,attributes_description,attributes_notes,attributes_department,...,attributes_url,attributes_new,attributes_services_count,attributes_cx_maturity_mapping_value,attributes_impact_mapping_value,attributes_service_provider_managers_0_email,attributes_service_provider_managers_0_first_name,attributes_service_provider_managers_0_last_name,attributes_service_provider_managers_0_position_title,attributes_service_provider_managers_0_profile_photo
0,36,service_providers,2098,USDA,Department of Agriculture,Farm Service Agency,usda-fsa,The Farm Service Agency (FSA) serves all farme...,,usda,...,None,False,2,4,30,None,NaN,NaN,NaN,NaN
1,28,service_providers,2098,USDA,Department of Agriculture,Food and Nutrition Service,usda-fns,Food and Nutrition Service (FNS) works to end ...,,Recertifying WIC participation,...,None,True,2,7,25,None,NaN,NaN,NaN,NaN


In [172]:
# mapping service desc
omb_df['service_desc'] = omb_df['attributes_service_id'].map(str).map(lambda x: serviceid2desc.get(x ))

# mapping service provider desc
omb_df['service_provider_desc'] = omb_df['attributes_service_provider_id'].map(lambda x: serviceproviderid2desc.get(str(x))
                                                                              )
display(omb_df.sample(2))

,id,type,attributes_organization_id,attributes_organization_name,attributes_organization_abbreviation,attributes_service_provider_id,attributes_service_provider_name,attributes_collection_id,attributes_collection_name,attributes_collection_year,...,attributes_q11_5,attributes_q11_point_scale,attributes_created_at,attributes_updated_at,attributes_operational_metrics,attributes_service_id,attributes_service_name,attributes_service_slug,service_desc,service_provider_desc
352,532,omb_cx_reporting_collections,2135,Department of the Interior,DOI,37,Fish and Wildlife Service,201,CX Quarterly Reporting,2022,...,0,NaN,2022-10-03T13:03:31.988Z,2022-10-03T13:03:31.988Z,,99,Planning a trip to a national wildlife refuge ...,,Customers of the U.S. Fish and Wildlife websit...,The U.S. Fish and Wildlife Service is the prem...
106,194,omb_cx_reporting_collections,2096,Department of Housing and Urban Development,HUD,11,Housing and Urban Development,59,CX Quarterly Reporting,2021,...,0,NaN,2021-07-02T21:52:01.975Z,2021-08-04T19:31:29.692Z,,15,Seeking information about FHA loan programs an...,hud-fha,"Trying to buy, maintain, keep and/or sell a ho...","In 2020, the Department of Housing and Urban D..."


In [173]:
omb_df.to_csv(f"./mergedcxdata-{today}.csv", index=False)